# Documentation:
help: https://docs.kepler.gl/docs/keplergl-jupyter#config

KeplerGL cannot read daskdataframe, so we saved and reloaded it as a pandas dataframe. Probably, if we define the metadata for the dask dataframe, the problem will be solved. CHECK IT!

The config file contains the configuration settings for this code only. We can load the map using "map = KeplerGl()", then "map (for ploting)", then chenge the view, add data using map.add_data(), set the desired filters, and finally running "map.config" for seeing the current, desired configurations. We can save the config or just copy past it into a config variable and pass this variable into KeplerGl().

# Preparing the data for KeplerGL (USing Dask)

In [1]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=6, 
                       threads_per_worker=2,
                       memory_limit='4GB')
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:40071 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 12 Memory: 24.00 GB


In [2]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
data = dd.read_csv('/home/saeedrmd/Documents/Work/Mobile Data/Run Data/Sample3000_5days.csv',parse_dates=['Date_Time'])

In [4]:
data['Hour'] = data.Date_Time.dt.hour

In [5]:
data_filteredbyH = data[(data.Hour>=6) & (data.Hour<=24)]

In [6]:
data_gbyH = data_filteredbyH.groupby('Hour')

In [7]:
pd.reset_option('all')


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/home/saeedrmd/anaconda3/envs/jupyter_env/lib/python3.8/site-packages/pandas/_config/config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [8]:
data.head()

,Date_Time,ID,Lat,Long,Hour
0,2019-08-03 03:00:04,35304,35.756,51.227,3
1,2019-08-03 03:00:04,35850,35.726,51.583,3
2,2019-08-03 03:00:14,27966,35.791,51.487,3
3,2019-08-03 03:00:17,31910,35.679,51.492,3
4,2019-08-03 03:00:18,31910,35.679,51.492,3


In [9]:
def custom_join(df):
    
    shp1 = gpd.read_file('/home/saeedrmd/Documents/Work/Mobile Data/TehranGeorefed/mahalat/mahalat.shp') #send this for mr Rahmani
    shp = shp1.to_crs({'init': 'epsg:4326'})
    geodf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df.Long, y=df.Lat), crs={'init': 'epsg:4326'})
    joined_data =gpd.sjoin(geodf, shp, how='left',op='within')
    joined_data = pd.DataFrame(joined_data)
    output = joined_data.groupby('index_right')['REGION'].count().reset_index()
    
    return output

In [10]:
countPerMahale = data_gbyH.apply(custom_join,meta={'index_right':np.int64,'REGION':np.int64}).compute()

In [11]:
shp1 = gpd.read_file('/home/saeedrmd/Documents/Work/Mobile Data/TehranGeorefed/mahalat/mahalat.shp')
shp = shp1.to_crs({'init': 'epsg:4326'})
Hours ={}
shp['index_right'] = range(0,353)
for i in range(6,24):
    Merged_shp = shp.merge(countPerMahale.loc[i],on='index_right',how='outer')
    Merged_shp.fillna(0,inplace=True)
    Hours['{}'.format(i)]=Merged_shp


/home/saeedrmd/anaconda3/envs/jupyter_env/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [12]:
l=[]
for i in range(6,24):
    shp['Hour {} Activity'.format(i)] = Hours['{}'.format(i)]['REGION_y']
    l.append('Hour {} Activity'.format(i))
    

==================================================================================================================
# Implementing Keplergl

In [13]:
from keplergl import KeplerGl

In [14]:
#this is for keplergl
ll=[]
for i in range(6,24):
    ddf=Hours['{}'.format(i)] 
    ddf['Time'] = pd.to_datetime('2019 {}:00:00'.format(i))
    ll.append(ddf)
dd_keplergl = pd.concat(ll).reset_index()
dd_keplergl.to_csv('/home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/kepler_csv_1.csv')

In [15]:
df = pd.read_csv('/home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/kepler_csv_1.csv')

# Method 1

In [ ]:
# Change to map as you want
# map_1.config
# save or copy the current config
map_1 = KeplerGl(height=800)
map_1.add_data(data=df, name='data_1')
# Change to map as you want (Defining the config)
# map_1.config
# Copy paste the values into the next variable (config)

config = {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['data_1'],
     'id': '8a5i98309',
     'name': ['Time'],
     'type': 'timeRange',
     'value': [1546343718000, 1546347197000],
     'enlarged': True,
     'plotType': 'histogram',
     'yAxis': None}],
   'layers': [{'id': '455z94m',
     'type': 'geojson',
     'config': {'dataId': 'data_1',
      'label': 'new layer',
      'color': [183, 136, 94],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': None,
       'colorRange': {'name': 'ColorBrewer YlOrRd-6',
        'type': 'sequential',
        'category': 'ColorBrewer',
        'colors': ['#ffffb2',
         '#fed976',
         '#feb24c',
         '#fd8d3c',
         '#f03b20',
         '#bd0026']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 22.4,
       'stroked': True,
       'filled': True,
       'enable3d': True,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'REGION_y', 'type': 'integer'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': {'name': 'REGION_y', 'type': 'integer'},
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'data_1': [{'name': 'Unnamed: 0',
        'format': None},
       {'name': 'index', 'format': None},
       {'name': 'REGION_x', 'format': None},
       {'name': 'NAME_MAHAL', 'format': None},
       {'name': 'CODE', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 24,
   'dragRotate': True,
   'latitude': 35.73926429865411,
   'longitude': 51.23263549178956,
   'pitch': 50,
   'zoom': 9.425885617522464,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [ ]:
# Add the defined config to the map
map_1.config = config

In [18]:
# Ploting the map
map_1

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['data_1'], 'id': '8a5i98309'…

In [19]:
# Saving the map using two methods
map_1.save_to_html(file_name='/home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/first_map_1.html')
# map_1.save_to_html(data={'data_1': df}, config=config, file_name='/home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/first_map_with_data.html')


Map saved to /home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/first_map_1.html!


# Method 2


If want to load the map next time with this saved config, the easiest way to do is to save it to a file 
and use the magic command %run to load it w/o cluttering up your notebook. See below and also the documentation
guide at the beginning of the code.

Example:

In [20]:
### Save map_1 config to a file:
# with open('/home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/config.py', 'w') as f:
#   f.write('config = {}'.format(map_1.config))

### load the config
%run /home/saeedrmd/Documents/Work/Mobile Data/KeplerGL/config.py (config is a presaved file)


Exception: File `'/home/saeedrmd/Documents/Work/Mobile.py'` not found.